# Introduction
## Finding Investment Properties in LA County
### There is a problem that is as old as property has been available for purchase to investors. This problem is, how can an investor know which property is the best one to purchase? There are many strategies to buying homes and many ways to be profitable. An investor could look for homes that are beat up, do some renovation, and sell the homes for a premium. An investor could look for homes that are underpriced and flip them. An investor could look for homes that have a low mortgage compared to how much rent could be brought in. And there are various other methods to achieving monetary gain. At the end of the day, the investor’s goal is to make money, and this is the problem we will address today.
### For this project we will focus on the investor who wants to buy a home that is an area that is going to gentrify. Gentrification is “the process of renovating and improving a house or district so that it conforms to middle-class taste” according to google. If an investor can find a house that is in an area that will gentrify, then they can buy it at a certain value, rent it for a few years, and then sell it at a much higher price when the area has improved. However, finding a neighborhood as it is undergoing these changes is a little more difficult than it may seem.
### The purpose of this project will be to determine which homes in LA County are likely to undergo gentrification. By defining certain neighborhood types according to the location data, it will be possible to correlate different neighborhoods that may be undergoing a change. These certain neighborhoods should contain a similar types of stores in near equivalent proportions. We will take a base neighborhood that is undergoing gentrification, like Long Beach, and then use a clustering algorithm to try and determine other neighborhoods that are undergoing similar changes. This will help the investor to know which areas of LA County to focus on. In a city with over 10 million people, finding the right neighborhoods is a great step for the investor.

# Data
## Data Sources:
### Location data for different cities within LA County. This can be found here: http://www.laalmanac.com/geography/ge09.php. This data will be used to segregate LA County into different sections. This will allow us to do analysis by groupings later. Different areas have different values and cultures so it is important that we capture this.
### Foursquare data. Using this data we will be able to see the kinds of establishments that are in areas that are gentrifying. The culture in an area that is gentrifying will be much different than an area that has already reached the improved state. Think of the difference between the kinds of stores you would find in Beverly Hills vs. Inglewood. There will no doubt be a noticeable difference. We are looking for the telltale signs that a neighborhood is about to gentrify or is already in the process.